In [2]:
Alphabet = ['A','T','G','C','$']
def rankAll(bwt):
    cc={}
    C = {}
    rankall={}
    for c in Alphabet:
        cc[c] = 0
        rankall[c] =[]
        rankall[c].append(0)
    for c in bwt :
        cc[c] = cc[c]+1
        for c in cc.iterkeys():
            rankall[c].append(cc[c])
    for base in Alphabet:
        C[base] = 0
        for c in Alphabet:
            if c < base:
                C[base] += cc[c]
    return rankall, cc, C

In [3]:
#seq = 'ATTATTAATTATATTATAATATA'
bwt =   'ATTTTTATTTA$AATATTTAAAAA'
r_bwt = 'ATTTT$TATTTATATATTAAAAAA'

"""
path1 = "/Users/Chen/Downloads/hu-chr1-bwts/hu-chr1.bwt.txt"
path2 = "/Users/Chen/Downloads/hu-chr1-bwts/hu-chr1-rev.bwt.txt"
rdFile1 = open(path1, "r")
bwt = rdFile1.read()
rdFile1.close()
rdFile2 = open(path2, "r")
r_bwt = rdFile2.read()
rdFile2.close()
"""

N = len(bwt)
print N
rankall, cc, C = rankAll(bwt)
r_rankall, r_cc, r_C = rankAll(r_bwt)
print rankall, cc, C
print "file loaded"

24
{'A': [0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 5, 5, 6, 6, 6, 6, 7, 8, 9, 10, 11], 'C': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'T': [0, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 8, 8, 8, 8, 9, 9, 10, 11, 12, 12, 12, 12, 12, 12], 'G': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], '$': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} {'A': 11, 'C': 0, 'T': 12, 'G': 0, '$': 1} {'A': 1, 'C': 12, 'T': 12, 'G': 12, '$': 0}
file loaded


In [4]:
def extendLeft (((i,j), (r_i,r_j)), c):
    ic = C[c] + rankall[c][i-1] + 1 
    jc = C[c] + rankall[c][j] 
    r_ic = r_i
    for base in Alphabet:
        if base < c:
            r_ic = r_ic + rankall[base][j] - rankall[base][i-1]
    r_jc = r_ic + (jc - ic)
    return (ic,jc), (r_ic, r_jc)

In [5]:
def extendRight ( ( (i,j), (r_i,r_j) ) , c):
    r_ic = C[c] + r_rankall[c][r_i - 1] + 1
    r_jc = C[c] + r_rankall[c][r_j]
    ic = i
    for base in Alphabet:
        if base < c:
            ic = ic + r_rankall[base][r_j] - r_rankall[base][r_i-1] 
            #print base, r_rankall[base][j] - r_rankall[base][i-1] , ic
    jc = ic + (r_jc - r_ic)
    return ( (ic,jc) , (r_ic,r_jc) )

In [6]:
def isValid (( (i,j), (r_i,r_j) )):
    return j >= i and r_j >= r_j
            
def isRightMaximal( ((i,j) , (r_i,r_j)) ):
    A = extendRight( ( (i,j) , (r_i,r_j)) , 'A' )
    C = extendRight( ( (i,j) , (r_i,r_j)) , 'C' )
    G = extendRight( ( (i,j) , (r_i,r_j)) , 'G' )
    T = extendRight( ( (i,j) , (r_i,r_j)) , 'T' )
    Dollar = extendRight( ( (i,j) , (r_i,r_j)) , '$')
    #print A , T, Dollar
    L = [isValid(A),isValid(T),isValid(C),isValid(G),isValid(Dollar)]
    return sum(L) >= 2

#print isRightMaximal(((21, 21), (4, 4)))

In [8]:
def stackKey(( (i,j) , (r_i,r_j) ) ):
    return max((j-i),(r_j - r_i))

def markKmer(k,n):
    stack = []
    stack.append( (( (1,n) , (1,n) ) , 0 ) )
    B = [0] * (n+1)
    count = 0
    while stack:
        (intervals, depth) = stack.pop()
        if depth < k:
            for c in Alphabet:
                ((i, j), (r_i, r_j))= extendRight( intervals , c )
                if j >= i:
                    B[i] = 1
        tempstack = []
        for c in Alphabet:
            intervals_left = extendLeft( intervals , c )
            if isRightMaximal(intervals_left) :
                tempstack.append( ( intervals_left , depth + 1 ) )
        tempstack = sorted(tempstack, key = lambda x: stackKey(x[0]), reverse = True)
        stack.extend(tempstack)

    return sum(B) - k

for k in range(1,24):
    print k, markKmer(k, N)

1 2
2 4
3 6
4 10
5 15
6 17
7 17
8 16
9 15
10 14
11 13
12 12
13 11
14 10
15 9
16 8
17 7
18 6
19 5
20 4
21 3
22 2
23 1
